In [1]:
!pip install mysql-connector-python

     ---------------------------------------- 0.0/527.0 kB ? eta -:--:--
     ---------------------------------------- 0.0/527.0 kB ? eta -:--:--
      --------------------------------------- 10.2/527.0 kB ? eta -:--:--
     ---- -------------------------------- 61.4/527.0 kB 812.7 kB/s eta 0:00:01
     ------------------ ------------------- 256.0/527.0 kB 2.2 MB/s eta 0:00:01
     -------------------------------------  522.2/527.0 kB 3.3 MB/s eta 0:00:01
     -------------------------------------- 527.0/527.0 kB 3.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/14.2 MB ? eta -:--:--
   - -------------------------------------- 0.4/14.2 MB 12.5 MB/s eta 0:00:02
   -- ------------------------------------- 0.8/14.2 MB 9.5 MB/s eta 0:00:02
   --- ------------------------------------ 1.3/14.2 MB 10.3 MB/s eta 0:00:02
   ----- ---------------------------------- 1.8/14.2 MB 10.4 MB/s eta 0:00:02
   ------- -------------------------------- 2.5/14.2 MB 11.4 MB/s eta 0:00:02
 

In [6]:
import mysql.connector
from mysql.connector import Error

def execute_sql_file(filename, host, database, user, password):
    # 데이터베이스 연결
    try:
        connection = mysql.connector.connect(host=host,
                                             database=database,
                                             user=user,
                                             password=password)
        if connection.is_connected():
            cursor = connection.cursor()

            # SQL 파일 읽기
            with open(filename, 'r') as file:
                sql_script = file.read()

            # SQL 명령문 실행
            for statement in sql_script.split(';'):
                if statement.strip():
                    cursor.execute(statement)

            connection.commit()
            print(f"SQL script '{filename}' executed successfully")

    except Error as e:
        print(f"Error while connecting to MySQL: {e}")

    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection is closed")

# 사용 예
execute_sql_file("../backup/db/study_20231228_172556.sql", "localhost", "study_test", "root", "0000")


SQL script '../backup/db/study_20231228_172556.sql' executed successfully
MySQL connection is closed


In [3]:
import pymysql
import os

def execute_sql_file(filepath, host, database, user, password):
    # MySQL 연결 설정
    connection = pymysql.connect(host=host,
                             user=user,
                             password=password,
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

    try:
        with connection.cursor() as cursor:
            # 'test' 데이터베이스 존재 여부 확인
            cursor.execute("SHOW DATABASES")
            databases = [db['Database'] for db in cursor.fetchall()]
            if database not in databases:
                # 'test' 데이터베이스가 없으면 생성
                cursor.execute(f"CREATE DATABASE {database} CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci")

        # 변경사항 커밋
        connection.commit()

    finally:
        connection.close()
        
    connection = pymysql.connect(host=host,
                                user=user,
                                password=password,
                                database=database,
                                charset='utf8mb4',
                                cursorclass=pymysql.cursors.DictCursor)
    try:
        with connection.cursor() as cursor:
            # 새 데이터베이스 스키마에서 추가된 컬럼 확인
            cursor.execute("DESCRIBE your_table_name")
            columns = [column['Field'] for column in cursor.fetchall()]

            # 백업된 SQL 파일 로드
            with open(filepath, 'r') as file:
                sql_file = file.read()
            
            # SQL 파일을 쿼리로 분리
            queries = sql_file.split(';')

            for query in queries:
                if 'INSERT INTO' in query:
                    # 삽입 쿼리 수정: 추가된 컬럼에 기본값 설정
                    query_parts = query.split('VALUES')
                    if len(query_parts) == 2:
                        insert_part, values_part = query_parts
                        new_values_part = []
                        for value in values_part.split('),'):
                            # 각 값 튜플에 대해 추가된 컬럼 수만큼 기본값 추가
                            new_values_part.append(value + ', DEFAULT' * (len(columns) - value.count(',')))
                        modified_query = insert_part + 'VALUES' + '),'.join(new_values_part)
                        cursor.execute(modified_query)

        # 변경사항 커밋
        connection.commit()

    finally:
        connection.close()

execute_sql_file("../backup/db/study_20231228_172556.sql", "localhost", "test_1", "root", "0000")

ProgrammingError: (1146, "Table 'test_1.your_table_name' doesn't exist")

In [8]:
import pymysql
import re

def get_db_columns(host, user, password, database, table):
    """ 데이터베이스에서 테이블의 컬럼 목록을 가져옵니다. """
    connection = pymysql.connect(host=host, user=user, password=password, database=database,
                                 cursorclass=pymysql.cursors.DictCursor)  # cursorclass 추가
    try:
        with connection.cursor() as cursor:
            cursor.execute(f"DESCRIBE {table}")
            return [row["Field"] for row in cursor.fetchall()]
    finally:
        connection.close()

def get_sql_file_columns(file_path):
    """ SQL 파일에서 모든 'CREATE TABLE' 문을 찾아 각 테이블의 컬럼 목록을 가져옵니다. """
    with open(file_path, 'r') as file:
        sql_file = file.read()

    # 모든 'CREATE TABLE' 문을 찾습니다.
    create_table_matches = re.findall(r'CREATE TABLE [^\(]+\(([^\)]+)', sql_file, re.IGNORECASE)

    columns = set()
    for create_table_content in create_table_matches:
        # 각 'CREATE TABLE' 문에서 컬럼 이름만 추출
        table_columns = re.findall(r'`(\w+)`', create_table_content)
        columns.update(table_columns)

    return columns

# 데이터베이스 및 테이블 정보
host = "localhost"
user = "root"
password = "0000"
database = "test_1"
table = "user"  # 데이터베이스의 테이블 이름

# SQL 파일 경로
sql_file_path = "../backup/db/study_20231228_172556.sql"

# 컬럼 목록 가져오기
db_columns = set(get_db_columns(host, user, password, database, table))
sql_file_columns = set(get_sql_file_columns(sql_file_path))

print(db_columns)
print(sql_file_columns)

# 차이점 출력
print("컬럼 (DB에만 존재):", db_columns - sql_file_columns)
print("컬럼 (SQL 파일에만 존재):", sql_file_columns - db_columns)


{'phone_number', 'state', 'profile_image', 'employment', 'user_name', 'email', 'password', 'user_id', 'profile_color', 'admin', 'use', 'attendance_type'}
{'post_id', 'schedule_id', 'comment_id', 'id', 'user_id'}
컬럼 (DB에만 존재): {'phone_number', 'state', 'profile_image', 'employment', 'user_name', 'email', 'password', 'profile_color', 'admin', 'use', 'attendance_type'}
컬럼 (SQL 파일에만 존재): {'id', 'post_id', 'schedule_id', 'comment_id'}


In [13]:
def get_sql_file_columns(file_path):
    with open(file_path, 'r') as file:
        sql_file = file.read()
    print(sql_file)
    create_table_matches = re.findall(r'CREATE TABLE [^\(]+\(([^\)]+)', sql_file, re.IGNORECASE)
    # print(create_table_matches)

sql_file_path = "../backup/db/study_20231228_172556.sql"
get_sql_file_columns(sql_file_path)

-- MySQL dump 10.13  Distrib 8.0.35, for Win64 (x86_64)
--
-- Host: 221.163.19.218    Database: study
-- ------------------------------------------------------
-- Server version	8.0.35-0ubuntu0.20.04.1

/*!40101 SET @OLD_CHARACTER_SET_CLIENT=@@CHARACTER_SET_CLIENT */;
/*!40101 SET @OLD_CHARACTER_SET_RESULTS=@@CHARACTER_SET_RESULTS */;
/*!40101 SET @OLD_COLLATION_CONNECTION=@@COLLATION_CONNECTION */;
/*!50503 SET NAMES utf8mb4 */;
/*!40103 SET @OLD_TIME_ZONE=@@TIME_ZONE */;
/*!40103 SET TIME_ZONE='+00:00' */;
/*!40014 SET @OLD_UNIQUE_CHECKS=@@UNIQUE_CHECKS, UNIQUE_CHECKS=0 */;
/*!40014 SET @OLD_FOREIGN_KEY_CHECKS=@@FOREIGN_KEY_CHECKS, FOREIGN_KEY_CHECKS=0 */;
/*!40101 SET @OLD_SQL_MODE=@@SQL_MODE, SQL_MODE='NO_AUTO_VALUE_ON_ZERO' */;
/*!40111 SET @OLD_SQL_NOTES=@@SQL_NOTES, SQL_NOTES=0 */;

--
-- Table structure for table `attendance`
--

DROP TABLE IF EXISTS `attendance`;
/*!40101 SET @saved_cs_client     = @@character_set_client */;
/*!50503 SET character_set_client = utf8mb4 */;
CREA

In [1]:
import pymysql

def create_database(connection, database_name):
    """ 새 데이터베이스 생성 """
    with connection.cursor() as cursor:
        cursor.execute(f"CREATE DATABASE IF NOT EXISTS {database_name}")
    connection.commit()

def get_tables_and_columns(connection, database_name):
    """ 데이터베이스의 테이블과 컬럼 구조 가져오기 """
    tables_and_columns = {}
    with connection.cursor() as cursor:
        cursor.execute(f"USE {database_name}")
        cursor.execute("SHOW TABLES")
        tables = [table[0] for table in cursor.fetchall()]
        for table in tables:
            cursor.execute(f"DESCRIBE {table}")
            columns = [column[0] for column in cursor.fetchall()]
            tables_and_columns[table] = columns
    return tables_and_columns

def create_tables_in_new_db(target_connection, database_name, tables_and_columns):
    """ 새 데이터베이스에 테이블과 컬럼 구조 생성 """
    with target_connection.cursor() as cursor:
        cursor.execute(f"USE {database_name}")
        for table, columns in tables_and_columns.items():
            # 테이블이 이미 존재하는지 확인
            cursor.execute(f"SHOW TABLES LIKE '{table}'")
            if cursor.fetchone() is None:
                # 테이블이 존재하지 않으면 생성
                column_definitions = ', '.join([f"`{column}` TEXT" for column in columns])
                cursor.execute(f"CREATE TABLE {table} ({column_definitions})")
    target_connection.commit()


def copy_data_to_new_db(source_connection, target_connection, tables_and_columns):
    """ 기존 데이터베이스에서 데이터를 새 데이터베이스로 복사 """
    for table, source_columns in tables_and_columns.items():
        with source_connection.cursor() as source_cursor, target_connection.cursor() as target_cursor:
            # 타겟 데이터베이스에서 해당 테이블의 컬럼 목록을 가져옵니다.
            target_cursor.execute(f"DESCRIBE `{table}`")
            target_columns = [column[0] for column in target_cursor.fetchall()]
            
            # 소스와 타겟의 공통 컬럼만 선택합니다.
            common_columns = [col for col in source_columns if col in target_columns]

            column_names = ', '.join([f"`{column}`" for column in common_columns])
            source_cursor.execute(f"SELECT {column_names} FROM `{table}`")
            
            for row in source_cursor.fetchall():
                try:
                    target_cursor.execute(
                        f"INSERT INTO `{table}` ({column_names}) VALUES ({', '.join(['%s'] * len(common_columns))})",
                        row)
                except pymysql.err.IntegrityError as e:
                    if e.args[0] == 1062:  # 중복 기본 키 오류
                        continue  # 중복된 레코드는 무시
                    else:
                        raise  # 다른 종류의 IntegrityError는 다시 발생시킴
            target_connection.commit()


def reorder_tables(tables_and_columns, first_table):
    """ 'first_table'을 가장 먼저 배치하고 나머지 테이블을 그 뒤에 배치합니다. """
    ordered_tables = list(tables_and_columns.keys())
    if first_table in ordered_tables:
        ordered_tables.remove(first_table)
        ordered_tables = [first_table] + ordered_tables
    return {table: tables_and_columns[table] for table in ordered_tables}

host = "localhost"
user = "root"
password = "0000"
database_source = "study_test"
database_target = "test_1"

# MySQL 연결 설정
source_connection = pymysql.connect(host=host, user=user, password=password, db=database_source)
target_connection = pymysql.connect(host=host, user=user, password=password)

# 새 데이터베이스 생성
create_database(target_connection, database_target)

# 기존 데이터베이스의 테이블과 컬럼 구조 가져오기
tables_and_columns = get_tables_and_columns(source_connection, database_source)

# 새 데이터베이스에 테이블 생성
create_tables_in_new_db(target_connection, database_target, tables_and_columns)

# 데이터 복사 순서를 조정
# 예: 먼저 'user' 테이블의 데이터를 복사한 후 'attendance' 테이블의 데이터를 복사
ordered_tables = ['user', 'attendance', 'schedules', 'free_board', 'fb_comment']  # 참조 순서에 따라 테이블 명을 조정하세요.
ordered_tables_and_columns = {table: tables_and_columns[table] for table in ordered_tables if table in tables_and_columns}

# 데이터 복사
copy_data_to_new_db(source_connection, target_connection, ordered_tables_and_columns)

# 연결 종료
source_connection.close()
target_connection.close()


# 데이터 베이스 백업 and 로드

In [4]:
import pymysql
from dotenv import load_dotenv
import os

load_dotenv()
DB_USER = os.getenv('MYSQL_USER')
DB_PASSWORD = os.getenv('MYSQL_PASSWORD')
DB_HOST = os.getenv('MYSQL_HOST')
DB_PORT = os.getenv('MYSQL_PORT')
DATABASE = os.getenv('MYSQL_DATABASE')
print(DB_HOST)
print(DATABASE)

backup_database = 'study_backup'
sql_file_path = "../backup/db/study_20240105_132100.sql"

localhost
study2


In [5]:
import mysql.connector
from mysql.connector import Error

def execute_sql_file(filename, database, host, user, password, port):
    # 데이터베이스 연결 (데이터베이스를 지정하지 않고 연결)
    try:
        connection = mysql.connector.connect(host=host,
                                             user=user,
                                             password=password,
                                             port=port)
        if connection.is_connected():
            cursor = connection.cursor()

            # 데이터베이스가 존재하는지 확인하고, 있으면 삭제
            cursor.execute(f"DROP DATABASE IF EXISTS {database}")

            # 새 데이터베이스 생성
            cursor.execute(f"CREATE DATABASE {database}")

            # 새 데이터베이스 사용
            cursor.execute(f"USE {database}")

            # SQL 파일 읽기
            with open(filename, 'r') as file:
                sql_script = file.read()

            # SQL 명령문 실행
            for statement in sql_script.split(';'):
                if statement.strip():
                    cursor.execute(statement)

            connection.commit()
            print(f"SQL script '{filename}' executed successfully")

    except Error as e:
        print(f"Error while connecting to MySQL: {e}")

    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection is closed")

# 사용 예
execute_sql_file(sql_file_path, backup_database, DB_HOST, DB_USER, DB_PASSWORD, DB_PORT)


SQL script '../backup/db/study_20240105_132100.sql' executed successfully
MySQL connection is closed


In [6]:

def create_database(connection, database_name):
    """ 새 데이터베이스 생성 """
    with connection.cursor() as cursor:
        cursor.execute(f"CREATE DATABASE IF NOT EXISTS {database_name}")
    connection.commit()

def get_tables_and_columns(connection, database_name):
    """ 데이터베이스의 테이블과 컬럼 구조 가져오기 """
    tables_and_columns = {}
    with connection.cursor() as cursor:
        cursor.execute(f"USE {database_name}")
        cursor.execute("SHOW TABLES")
        tables = [table[0] for table in cursor.fetchall()]
        for table in tables:
            cursor.execute(f"DESCRIBE {table}")
            columns = [column[0] for column in cursor.fetchall()]
            tables_and_columns[table] = columns
    return tables_and_columns

def create_tables_in_new_db(target_connection, database_name, tables_and_columns):
    """ 새 데이터베이스에 테이블과 컬럼 구조 생성 """
    with target_connection.cursor() as cursor:
        cursor.execute(f"USE {database_name}")
        for table, columns in tables_and_columns.items():
            # 테이블이 이미 존재하는지 확인
            cursor.execute(f"SHOW TABLES LIKE '{table}'")
            if cursor.fetchone() is None:
                # 테이블이 존재하지 않으면 생성
                column_definitions = ', '.join([f"`{column}` TEXT" for column in columns])
                cursor.execute(f"CREATE TABLE {table} ({column_definitions})")
    target_connection.commit()


def copy_data_to_new_db(source_connection, target_connection, tables_and_columns):
    for table, source_columns in tables_and_columns.items():
        with source_connection.cursor() as source_cursor, target_connection.cursor() as target_cursor:
            target_cursor.execute(f"DESCRIBE `{table}`")
            target_column_details = target_cursor.fetchall()

            # 소스와 타겟의 공통 컬럼만 선택합니다.
            common_columns = [col for col in source_columns if col in [col_detail[0] for col_detail in target_column_details]]

            # 새로운 컬럼에 대한 기본값 설정
            default_values = {}
            for column_detail in target_column_details:
                column_name = column_detail[0]
                if column_name not in common_columns:
                    # 컬럼 타입에 따라 기본값 설정 (정수형의 경우 0, 그 외의 경우 빈 문자열)
                    if 'int' in column_detail[1]:
                        default_values[column_name] = 0
                    else:
                        default_values[column_name] = ''

            column_names = ', '.join([f"`{column}`" for column in common_columns + list(default_values.keys())])
            source_cursor.execute(f"SELECT {', '.join([f'`{column}`' for column in common_columns])} FROM `{table}`")
            
            for row in source_cursor.fetchall():
                row_data = list(row) + list(default_values.values())  # 기존 데이터에 새 컬럼의 기본값 추가
                try:
                    target_cursor.execute(
                        f"INSERT INTO `{table}` ({column_names}) VALUES ({', '.join(['%s'] * len(row_data))})",
                        row_data)
                except pymysql.err.IntegrityError as e:
                    if e.args[0] == 1062:  # 중복 기본 키 오류
                        continue  # 중복된 레코드는 무시
                    else:
                        raise  # 다른 종류의 IntegrityError는 다시 발생시킴
            target_connection.commit()

print(DATABASE)
# MySQL 연결 설정
source_connection = pymysql.connect(host=DB_HOST, user=DB_USER, password=DB_PASSWORD, db=backup_database)
target_connection = pymysql.connect(host=DB_HOST, user=DB_USER, password=DB_PASSWORD)

# 새 데이터베이스 생성
create_database(target_connection, DATABASE)

# 기존 데이터베이스의 테이블과 컬럼 구조 가져오기
tables_and_columns = get_tables_and_columns(source_connection, backup_database)

# 새 데이터베이스에 테이블 생성
create_tables_in_new_db(target_connection, DATABASE, tables_and_columns)

# 데이터 복사 순서를 조정
# 예: 먼저 'user' 테이블의 데이터를 복사한 후 'attendance' 테이블의 데이터를 복사
ordered_tables = ['user', 'attendance', 'schedules', 'free_board', 'fb_comment']  # 참조 순서에 따라 테이블 명을 조정하세요.
ordered_tables_and_columns = {table: tables_and_columns[table] for table in ordered_tables if table in tables_and_columns}

# 데이터 복사
copy_data_to_new_db(source_connection, target_connection, ordered_tables_and_columns)

# 연결 종료
source_connection.close()
target_connection.close()

study2
